Comparing bonus unemployment, payroll tax cuts, and universal payments
=============================================================

This uses the 2018 Current Population Survey March Supplement, Tax-Calculator, and the Supplemental Poverty Measure to estimate the effects of the Federal Pandemic Unemployment Compensation (extra 600 dollars per week), a budget-neutral payroll tax cuts, and a budget-neutral universal payment.

In [ ]:
import numpy as np
import pandas as pd
import microdf as mdf
import plotly.express as px
import plotly

spmu = pd.read_csv('spmu.csv.gz')
person = pd.read_csv('person.csv.gz')

# All potential reforms.
CHG_COLS = ['fpuc_net', 'fpuc_ubi', 'fpuc_adult_ubi', 'fpuc_fica_cut',
            'fpuc2_net', 'fpuc2_ubi', 'fpuc2_adult_ubi', 'fpuc2_fica_cut']

## Poverty analysis

In [ ]:
def pov(reform, year, age_group='All', race='All'):
    """ Calculate the poverty rate under the specified reform for the
        specified population.
        Note: All arguments refer to the poverty population, not the reform.
    
    Args:
        reform: One of CHG_COLS. If None, provides the baseline rate.
        year: Year of the data (year before CPS survey year).
        age_group: Age group, either
            - 'Children' (under 18)
            - 'Adults' (18 or over)
            - 'All'
        race: Race code to filter to. Defaults to 'All'.
        
    Returns:
        2018 SPM poverty rate.
    """
    if reform == 'baseline':
        resource_col = 'spmpoor'
    else:
        resource_col = 'spmpoor_' + reform
    target_persons = person[person.FLPDYR == year]
    if age_group == 'Children':
        target_persons = target_persons[target_persons.age < 18]
    elif age_group == 'Adults':
        target_persons = target_persons[target_persons.age >= 18]
    if race != 'All':
        target_persons = target_persons[target_persons.race == race]
    return mdf.weighted_mean(target_persons, resource_col, 'asecwt')

def pov_row(row):
    """ Calculate poverty based on parameters specified in the row.
    
    Args:
        row: pandas Series.
        
    Returns:
        2018 SPM poverty rate.
    """
    return pov(row.reform, row.age_group, row.race)

In [ ]:
pov_rates = mdf.cartesian_product({'reform': ['baseline'] + CHG_COLS,
                                   'year': person.FLPDYR.unique(),
                                   'age_group': ['All', 'Children', 'Adults'],
                                   'race': ['All', 200]})  # 200 means Black.
pov_rates['pov'] = 100 * pov_rates.apply(pov_row, axis=1)

Top-level poverty rates.

In [ ]:
pov_rates[(pov_rates.age_group == 'All') & (pov_rates.race == 'All')]

Top-level child poverty rates.

In [ ]:
pov_rates[(pov_rates.age_group == 'Children') & (pov_rates.race == 'All')]

### Poverty gap and inequality

Calculate these for all people and SPM units, without breaking out by age or race.

In [ ]:
def pov_gap_b(reform, year):
    if reform == 'baseline':
        resource_col = 'spmtotres'
    else:
        resource_col = 'spmtotres_' + reform
    spmu = spmu[spmu.FLPDYR == year]
    pov_gap = np.maximum(spmu.spmthresh - spmu[resource_col], 0)
    return (pov_gap * spmu.spmwt).sum() / 1e9

def pov_gap_row(row):
    return pov_gap_b(row.reform, row.year)

In [ ]:
pov_gap_ineq = pov_rates[['reform', 'year']].drop_duplicates()
pov_gap_ineq['pov_gap_b'] = pov_gap_ineq.apply(pov_gap_row, axis=1)

### Inequality

By individual based on their percentage of SPM resources.

In [ ]:
def gini(reform, year):
    if reform == 'baseline':
        resource_col = 'spmtotres'
    else:
        resource_col = 'spmtotres_' + reform
    tmp = person[person.FLPDYR == year]
    return mdf.gini(tmp[resource_col], tmp.asecwt)

def gini_row(row):
    return gini(row.reform, row.year)

In [ ]:
pov_gap_ineq['gini'] = pov_gap_ineq.apply(gini_row, axis=1)
pov_gap_ineq